## 미니 프로젝트
 - 숫자 퍼즐 게임 완성하기
 - 아래와 같이 숫자 퍼즐을 만들고 숫자를 이동시켜 순서대로 맞추는 게임
 ![퍼즐이미지](https://i.stack.imgur.com/0B14h.png)
 - [이미지 출처](https://math.stackexchange.com/questions/635188/what-is-the-parity-of-permutation-in-the-15-puzzle)

In [1]:
import random

### 게임 로직 구현하기
 1. 퍼즐 생성하기
 2. 퍼즐 랜덤하게 섞기
 3. 퍼즐 출력
 4. 사용자 입력(움직일 숫자 입력 받기)
 5. 퍼즐 완성 확인하기
   - 완성? 완료 메시지와 함께 종료
   - 미완성? 3번으로 이동

#### 퍼즐 생성하기
 - 2차원 리스트 형태로 생성
 - 퍼즐의 크기(size)를 파라미터로 받아, 동적으로 size*size의 리스트로 생성
 - 퍼즐이 생성되면 1부터 차례대로 행방향으로 숫자를 나열
   - 사이즈가 3인 경우의 생성 예
   -  [[1, 2, 3],
      [4, 5, 6],
      [7, 8, 9]]
 - 퍼즐의 가장 마지막 아이템(마지막 행의 마지막 열 아이템)은 '' 빈문자열로 처리
   - 이유는? 숫자퍼즐의 목표는 빈공간을 이용해 각 이동하고자 하는 숫자를 빈공간으로 움직여 숫자들을 순서대로 다시 맞추는 것이 목적이므로, 빈공간을 표현하기 위한 방법으로 빈문자열을 사용

* pure python 버젼

In [2]:
def initiate_puzzle(size):
    '''
    파라미터
     size: 퍼즐의 크기
    리턴
     생성된 퍼즐 리스트
    '''
    puzzle = []
    for i in range(size):
        row = []
        for j in range(size):
            row.append(i*size + j+1)
        puzzle.append(row)

    puzzle[-1][-1] = ''
    return puzzle

* numpy 버젼

In [3]:
import numpy as np
def initiate_puzzle(size):
    '''
    파라미터
     size: 퍼즐의 크기
    리턴
     생성된 퍼즐 리스트
    '''
    puzzle = np.arange(1, size*size+1).reshape(size, size)
    puzzle = puzzle.tolist() 
    puzzle[-1][-1] = ''
    return puzzle

In [4]:
puzzle = initiate_puzzle(4)

#### 퍼즐 출력하기
 - 생성된 퍼즐(puzzle)을 파라미터로 받아 화면에 출력
 - 이때, 퍼즐은 2차원 형태이므로 2중 loop을 이용

In [5]:
def show_puzzle(puzzle):
    '''
    파라미터
     puzzle: 퍼즐 
    리턴
     None
    '''
    for i in range(len(puzzle)):
        for j in range(len(puzzle[0])):
            print('{:3}'.format(puzzle[i][j]), sep=' ', end='')
        print()

In [6]:
show_puzzle(puzzle)

  1  2  3  4
  5  6  7  8
  9 10 11 12
 13 14 15   


#### 퍼즐 섞기(shuffling)
 - 생성할때부터 랜덤하게 숫자를 배열하지 않고, 완성된 상태에서 퍼즐을 섞어야 함
   - 이유? 랜덤하게 배열하는 경우, 퍼즐이 완성되지 못하는 경우의 수가 수학적으로 존재하기 때문
   - 퍼즐을 완성시킬 수 없는 예
   - [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 15, 14, '']]

In [7]:
def get_index(puzzle, n):
    '''
    파라미터
      puzzle: 퍼즐
      n: 퍼즐 내에서 찾으려는 숫자 혹은 빈공간('') 값
    리턴
      퍼즐에서 해당 숫자나 빈공간을 찾았다면 해당 인덱스를 반환
      찾지 못했다면 None, None 반환
    '''
    for i in range(len(puzzle)):
        try:
            index = puzzle[i].index(n)
            return i, index
        except:
            continue
    return None, None

In [8]:
def shuffle_puzzle(puzzle, shuffle_count):
    '''
    파라미터
     puzzle: 퍼즐
     shuffle_count: 섞을 횟수
    리턴
     None
    '''
    
    # 각각 섞을 때마다 빈공간을 기준으로 상하좌우의 방향으로 섞기 위해
    # 방향 리스트 생성
    # 순서대로 상 우 하 좌 
    dxs = [1, 0, -1,  0]
    dys = [0, 1,  0, -1]

    cnt = 0
    while cnt <= shuffle_count:
        rnd = random.randint(0, 3)
        dx = dxs[rnd]
        dy = dys[rnd]

        # 빈공간의 index 얻기
        i, j = get_index(puzzle, '')            
        ni = i + dx
        nj = j + dy

        # 새로 얻은 인덱스 확인(퍼즐 범위내에 포함되는지) 하여 숫자 이동
        if 0 <= ni < len(puzzle) and 0 <= nj < len(puzzle[0]):
            puzzle[ni][nj], puzzle[i][j] = puzzle[i][j], puzzle[ni][nj]
        
        cnt += 1

In [9]:
shuffle_puzzle(puzzle, 10)
show_puzzle(puzzle)

  1  2  3  4
  5  6  7   
  9 10 11  8
 13 14 15 12


#### 퍼즐이 완성되었는지 확인하기
 - 퍼즐이 완성된 형태인지 확인
 - puzzle 퍼즐로 활용할 리스트, completed 완성된 형태의 퍼즐 리스트 
 - 완성되었다면 True, 아니라면 False 반환

In [10]:
def is_puzzle_completed(puzzle, completed):
    return puzzle == completed

#### 퍼즐 이동하기
 - 퍼즐 내의 숫자를 이동
 - 이때 이동이 가능한 경우는 해당 숫자가 빈공간 상하좌우에 위치한 경우에만 가능

In [11]:
def move_by_number(puzzle, n):
    # 숫자가 위치한 index
    i, j = get_index(puzzle, n)
    
    # index를 이용하여 숫자 이동
    move_by_index(puzzle, i, j)

In [12]:
def move_by_index(puzzle, i, j):
    # 좌우위아래 한방향중 하나가 '' 값이라면 이동 가능
    for dx, dy in ((1, 0), (0, 1), (-1, 0), (0, -1)):
        new_i = i + dx
        new_j = j + dy

        # boundary 체크(갈 수 없는 곳이면 패스)
        if not (0 <= new_i < len(puzzle) and 0 <= new_j < len(puzzle[0])):
            continue

        # 옆에 빈 공간인 경우에는 퍼즐의 위치를 빈공간과 바꿈(swap)
        if puzzle[new_i][new_j] == '':
            puzzle[i][j], puzzle[new_i][new_j] = puzzle[new_i][new_j], puzzle[i][j]
            return 

#### 사용자 프롬프트 입력
 - 게임의 진행을 위해 동적으로 키보드 입력을 받을 필요가 있음
   - 퍼즐의 크기, 이동할 수 지정 
 - 이를 위해 input 함수 사용
   - 원하는 값 입력후, Enter

In [14]:
value = input('입력하세요')
print(value)

입력하세요4
4


* 입력받은 값을 숫자형태로 변경

In [15]:
value = int(input('숫자를 입력하세요'))
print(value)

숫자를 입력하세요5
5


#### 퍼즐, 퍼즐 완성본 생성 및 셔플링

* 퍼즐 사이즈 입력

In [16]:
size = int(input('-> 퍼즐 사이즈를 입력하세요: '))
print('퍼즐 사이즈: ', size)

-> 퍼즐 사이즈를 입력하세요: 5
퍼즐 사이즈:  5


* 퍼즐 생성

In [17]:
puzzle = initiate_puzzle(size)

* 퍼즐 완성본 생성
 - 기존 퍼즐을 복사하여 생성
 - 아래와 같이 deep copy본으로 생성
   - 그렇지 않으면, 항상 puzzle과 complete가 동일한 객체가 됨

In [18]:
complete = [row[:] for row in puzzle]

* copy 모듈을 이용해서도 가능

In [19]:
import copy
complete = copy.deepcopy(puzzle)

In [20]:
show_puzzle(complete)

  1  2  3  4  5
  6  7  8  9 10
 11 12 13 14 15
 16 17 18 19 20
 21 22 23 24   


In [21]:
show_puzzle(puzzle)

  1  2  3  4  5
  6  7  8  9 10
 11 12 13 14 15
 16 17 18 19 20
 21 22 23 24   


* 퍼즐 섞기

In [22]:
shuffle_puzzle(puzzle, 300)

In [23]:
show_puzzle(puzzle)

  6  1 10  9  3
  7 11  2 20  4
 17 22 16 14  5
     8 18 12 15
 13 21 23 19 24


#### 게임 루프 
 - 퍼즐이 완성되었나 확인
   - 완성되었다면 종료
   - 완성되지 않았다면 사용자 입력 대기 및 퍼즐 출력 

In [24]:
# output을 clear하기 위해 사용
from IPython.display import clear_output

In [ ]:
show_puzzle(puzzle)

while not is_puzzle_completed(puzzle, complete):
    # 숫자를 입력하지 않은 경우에 대한 예외 처리
    try:
        num = int(input(' -> 움직일 숫자를 입력하세요 : '))
    except:
        print('숫자가 아닙니다.')
        continue

    # 움직일 수 선택하기
    move_by_number(puzzle, num)

    # 화면 clear
    clear_output()

    # 움직인 이후 퍼즐 상태 보기
    show_puzzle(puzzle)

# 루프의 종료는 곧 퍼즐의 완성을 의미!
print('\n퍼즐 완성!')

  6  1 10  9  3
  7 11  2 20  4
 17 22 16 14  5
  8    18 12 15
 13 21 23 19 24
